# I. Collecting dataset

### Import các thư viện cần thiết

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Lấy tên các công ty được lưu trong `company_name.txt`

In [2]:
with open('../Data/company_name.txt', 'r') as file:
    companies = file.read().splitlines()

Kiểm tra request

In [3]:
responses = []

for company in companies[:1]:
    # Construct the URL for each company
    URL = f'https://companiesmarketcap.com/{company}/total-assets/'
    response = requests.get(URL)
    
    if response.ok:
        message = 'The request was successfully received.'
    else:
        message = 'The request was not successful.'
    
    responses.append(response)
    print(f"For {company}: {message}")

For apple: The request was successfully received, understood, and accepted.


Bây giờ, ta sẽ xem thử phần nội dung của `html repsonse`

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en-US">
<head>
<title>Apple (AAPL) - Total assets</title>
<meta charset="utf-8"/>
<link href="https://companiesmarketcap.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/img/icons/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/img/icons/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/img/icons/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/img/icons/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/img/icons/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/img/icons/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/img/icons/apple-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/img/icons/apple-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/img/icons/apple-icon-180x180.png" rel="apple-touch-icon" siz

Thấy được dữ liệu ta cần lấy nằm trong tag `tbody`

In [8]:
table = soup.find('tbody')
table.prettify

<bound method Tag.prettify of <tbody>
<tr><td>2023-09-30</td><td>$352.58 B</td><td class="percentage-red">-0.05%</td></tr><tr><td>2022-09-30</td><td>$352.75 B</td><td class="percentage-green">0.5%</td></tr><tr><td>2021-09-30</td><td>$351.00 B</td><td class="percentage-green">8.37%</td></tr><tr><td>2020-09-30</td><td>$323.88 B</td><td class="percentage-red">-4.32%</td></tr><tr><td>2019-09-30</td><td>$338.51 B</td><td class="percentage-red">-7.44%</td></tr><tr><td>2018-09-30</td><td>$365.72 B</td><td class="percentage-red">-2.56%</td></tr><tr><td>2017-09-30</td><td>$375.31 B</td><td class="percentage-green">16.67%</td></tr><tr><td>2016-09-30</td><td>$321.68 B</td><td class="percentage-green">10.74%</td></tr><tr><td>2015-09-30</td><td>$290.47 B</td><td class="percentage-green">25.29%</td></tr><tr><td>2014-09-30</td><td>$231.83 B</td><td class="percentage-green">12%</td></tr><tr><td>2013-09-30</td><td>$207.00 B</td><td class="percentage-green">17.57%</td></tr><tr><td>2012-09-30</td><td>$17

**Nhận xét**

Thấy thông tin mỗi năm nằm trong `tr` 
Trong đó , thông tin năm có định dạng `yyyy-mm-dd` và có những năm không có  
Do đó, ta sẽ tạo một dictionary với key là năm và value là thông tin

In [16]:
trs = table.find_all('tr')
data = {}  
years = set()
for tr in trs:
    td_elements = tr.find_all('td')
    if len(td_elements) == 3:
        year = td_elements[0].text.strip().split('-')[0] 
        years.add(year) 
        data[year] = td_elements[1].text.strip()
data

{'2023': '$352.58 B',
 '2022': '$352.75 B',
 '2021': '$351.00 B',
 '2020': '$323.88 B',
 '2019': '$338.51 B',
 '2018': '$365.72 B',
 '2017': '$375.31 B',
 '2016': '$321.68 B',
 '2015': '$290.47 B',
 '2014': '$231.83 B',
 '2013': '$207.00 B',
 '2012': '$176.06 B',
 '2011': '$116.37 B',
 '2010': '$75.18 B',
 '2009': '$53.85 B',
 '2008': '$39.57 B',
 '2007': '$25.34 B',
 '2006': '$17.20 B',
 '2005': '$11.55 B',
 '2004': '$8.05 B',
 '2003': '$6.81 B',
 '2002': '$6.29 B',
 '2001': '$6.02 B'}

Giờ ta sẽ làm y hệt toàn bộ công ti, và lưu dữ liệu vào trong `DataFrame`.  

Trước hết ta sẽ lấy dữ liệu về `Total assets`

In [18]:
all_company_data = []
years = set()  
for company in companies:
    URL = f'https://companiesmarketcap.com/{company}/total-assets/'
    response = requests.get(URL)

    if response.ok:
        print(company)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('tbody')

        if table:
            trs = table.find_all('tr')
            data = {}  

            for tr in trs:
                td_elements = tr.find_all('td')
                if len(td_elements) == 3:
                    year = td_elements[0].text.strip().split('-')[0]  
                    years.add(year)  
                    data[year] = td_elements[1].text.strip()

            all_company_data.append({'Company': company, **data})  


years_list = sorted(list(years), reverse=True)

total_assets_df = pd.DataFrame(all_company_data)
total_assets_df = total_assets_df.reindex(columns=['Company'] + years_list)

apple
microsoft
saudi-aramco
alphabet-google
amazon


In [21]:
total_assets_df.sample(5)

,Company,2023,2022,2021,2020,2019,2018,2017,2016,2015,...,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001
0,apple,$352.58 B,$352.75 B,$351.00 B,$323.88 B,$338.51 B,$365.72 B,$375.31 B,$321.68 B,$290.47 B,...,$75.18 B,$53.85 B,$39.57 B,$25.34 B,$17.20 B,$11.55 B,$8.05 B,$6.81 B,$6.29 B,$6.02 B
3,alphabet-google,NaN,$365.26 B,$359.26 B,$319.61 B,$275.90 B,$232.79 B,$197.29 B,$167.49 B,$147.46 B,...,$57.85 B,$40.49 B,$31.76 B,$25.33 B,$18.47 B,$10.27 B,$3.31 B,$0.87 B,$0.28 B,NaN
1,microsoft,$411.97 B,$364.84 B,$333.77 B,$301.31 B,$286.55 B,$258.84 B,$241.08 B,$193.69 B,$176.22 B,...,$86.11 B,$77.88 B,$72.79 B,$63.17 B,$69.59 B,$70.81 B,$92.38 B,$79.57 B,$67.64 B,$59.25 B
4,amazon,NaN,$462.67 B,$420.54 B,$321.19 B,$225.24 B,$162.64 B,$131.31 B,$83.40 B,$65.44 B,...,$18.79 B,$13.81 B,$8.31 B,$6.48 B,$4.36 B,$3.69 B,$3.24 B,$2.16 B,$1.99 B,$1.63 B
2,saudi-aramco,NaN,$663.34 B,$576.72 B,$510.78 B,$398.60 B,$358.83 B,$294.01 B,$250.76 B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lưu dữ liệu vào file `total_assets.csv`

In [ ]:
total_assets_df.to_csv('../Data/total_assets.csv', index=False)

Làm tương tự cho phần `debt`

In [ ]:
all_company_data = []
years = set()  
for company in companies:
    URL = f'https://companiesmarketcap.com/{company}/total-debt/'
    response = requests.get(URL)

    if response.ok:
        print(company)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('tbody')

        if table:
            trs = table.find_all('tr')
            data = {}  

            for tr in trs:
                td_elements = tr.find_all('td')
                if len(td_elements) == 3:
                    year = td_elements[0].text.strip().split('-')[0]  
                    years.add(year)  
                    data[year] = td_elements[1].text.strip()

            all_company_data.append({'Company': company, **data})  


years_list = sorted(list(years), reverse=True)

total_debt_df = pd.DataFrame(all_company_data)
total_debt_df = total_assets_df.reindex(columns=['Company'] + years_list)

Lưu dữ liệu vào file `debts.csv`

In [ ]:
total_debt_df.to_csv('../Data/debts.csv', index=False)

Làm tương tự cho phần `net assets`

In [ ]:
all_company_data = []
years = set()  
for company in companies:
    URL = f'https://companiesmarketcap.com/{company}/net-assets/'
    response = requests.get(URL)

    if response.ok:
        print(company)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('tbody')

        if table:
            trs = table.find_all('tr')
            data = {}  

            for tr in trs:
                td_elements = tr.find_all('td')
                if len(td_elements) == 3:
                    year = td_elements[0].text.strip().split('-')[0]  
                    years.add(year)  
                    data[year] = td_elements[1].text.strip()

            all_company_data.append({'Company': company, **data})  


years_list = sorted(list(years), reverse=True)

net_assets_df = pd.DataFrame(all_company_data)
net_assets_df = total_assets_df.reindex(columns=['Company'] + years_list)

Lưu dữ liệu vào file csv

In [ ]:
net_assets_df.to_csv('../Data/net_assets.csv', index=False)

Làm tương tự cho phần `cash on hand`

In [ ]:
all_company_data = []
years = set()  
for company in companies:
    URL = f'https://companiesmarketcap.com/{company}/cash-on-hand/'
    response = requests.get(URL)

    if response.ok:
        print(company)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('tbody')

        if table:
            trs = table.find_all('tr')
            data = {}  

            for tr in trs:
                td_elements = tr.find_all('td')
                if len(td_elements) == 3:
                    year = td_elements[0].text.strip().split('-')[0]  
                    years.add(year)  
                    data[year] = td_elements[1].text.strip()

            all_company_data.append({'Company': company, **data})  


years_list = sorted(list(years), reverse=True)

cash_on_hand_df = pd.DataFrame(all_company_data)
cash_on_hand_df = total_assets_df.reindex(columns=['Company'] + years_list)

cash_on_hand_df.to_csv('../Data/cash_on_hand.csv', index=False)